In [1]:
import base64
import os
import requests

This notebook shows some examples of how to call a model trained and deployed in the Peltarion platform from Python using JSON requests.

The advantage of using JSON is that you can score several examples in a batch rather than one at a time as with curl or POST requests with forms.

## Image classification (single input, single output)

We will hide the deployment URL and the authentication token for security reasons. You can substitute the values found on the Deployment page for the model we want to use.

The following example assumes that you have a model trained on MNIST data (28x28 pixels, 3 color channels) and want to classify new images that the model has not seen.

In [2]:
url = ' -- insert the URL you find on the deployment page -- '
token = ' -- insert the token you find on the deployment page --'

The file called `three.png` can be found in the images folder of this repo.

In order to feed the image to the deployment API, we need to encode it in base64 format and prepend a short string explaining what type of data it is.

In [3]:
img_file = "images/three.png"
img_type = os.path.splitext(img_file)[-1][1:]
with open(img_file, "rb") as image_file:
    encoded_img = 'data:image/{};base64,'.format(img_type) + base64.b64encode(image_file.read()).decode('ascii')

The structure of the JSON string that we will send is shown below. The "rows" key must always be present and its value is a list of strings that represent different examples that we want to classify. Each entry of the list contains a comma-separated set of key-value pairs where the key is the feature name and the value is the feature value (which will be a base64 string for images.)

```
  {"rows": 
  	[{"feature1": "value1", "feature2": "value2"}, 
  	 {"feature1": "value1", "feature2": "value2"}
  	]
  }
```
For the current example, we only have one example and one feature called "Image". (You can find the feature names on the deployment page.) The structure of the JSON will be fairly simple:

```
  {"rows": 
  	[{"Image": "<base64 encoded image>"}]
  }
```

In [4]:
payload = "{\"rows\": [{\"Image\":\"" + encoded_img + "\"}]}"
headers = {
    'Content-Type': "application/json",
    'Authorization': "Bearer {}".format(token),
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.json())

{'rows': [{'Number': {'0': 1.172899e-12, '1': 1.904032e-23, '2': 1.0426288e-09, '3': 0.99999976, '4': 8.7458367e-32, '5': 2.2160087e-07, '6': 1.1102806e-25, '7': 8.750066e-19, '8': 9.540449e-19, '9': 6.100877e-09}}]}


### Classify several images

To simplify our life, we might want to write a small function that encodes an image to base64 given a file path.

In [5]:
def encode_img(path):
    img_type = os.path.splitext(path)[-1][1:]
    with open(path, "rb") as image_file:
        encoded_img = 'data:image/{};base64,'.format(img_type) + base64.b64encode(image_file.read()).decode('ascii')
    return encoded_img

Now we can classify a batch of images (in this case just two, but it would work with a much larger batch too.)

In [7]:
img_files = ['images/three.png', 'images/Six.png']
encoded_imgs = [encode_img(f) for f in img_files]
input_batch = ','.join(["{\"Image\":\"" + encoded_img + "\"}" for encoded_img in encoded_imgs])
payload = "{\"rows\": [" + input_batch + "]}"

In [8]:
response = requests.request("POST", url, data=payload, headers=headers)
response.json()

{'rows': [{'Number': {'0': 1.1729013e-12,
    '1': 1.904032e-23,
    '2': 1.0426308e-09,
    '3': 0.99999976,
    '4': 8.7458367e-32,
    '5': 2.2160044e-07,
    '6': 1.1102806e-25,
    '7': 8.750066e-19,
    '8': 9.540412e-19,
    '9': 6.100854e-09}},
  {'Number': {'0': 9.634732e-05,
    '1': 8.909446e-09,
    '2': 3.8414715e-08,
    '3': 1.0719662e-05,
    '4': 7.3074455e-07,
    '5': 0.0041560247,
    '6': 0.9956968,
    '7': 8.453612e-08,
    '8': 1.8743322e-05,
    '9': 2.0451373e-05}}]}

## Tabular data (multiple numeric inputs)

This example assumes that we have trained a model on the California housing dataset, where we try to predict latitude from some numeric features.

In [9]:
url = ' -- insert the URL you find on the deployment page -- '
token = ' -- insert the token you find on the deployment page --'

We can define a short utility function to construct a row for a training example in the right format.

In [16]:
def input_row(input_params):
    return '{' + ','.join(["\"" + name + "\":" + value for (name, value) in input_params.items()]) + '}'

In [17]:
ex1 = {
"population": "1551.0",
"totalBedrooms": "434.0",
"totalRooms": "2202.0",
"housingMedianAge": "52.0",
"medianHouseValue": "261100.0",
"medianIncome": "3.12",
"households": "514.0"
}

ex2 = {
"population": "3551.0",
"totalBedrooms": "834.0",
"totalRooms": "2902.0",
"housingMedianAge": "76.0",
"medianHouseValue": "111100.0",
"medianIncome": "2.12",
"households": "1000.0"
}

examples = [ex1, ex2]
input_batch = ','.join([input_row(ex) for ex in examples])
payload = "{\"rows\": [" + input_batch + "]}"

In [18]:
payload

'{"rows": [{"population":1551.0,"totalBedrooms":434.0,"totalRooms":2202.0,"housingMedianAge":52.0,"medianHouseValue":261100.0,"medianIncome":3.12,"households":514.0},{"population":3551.0,"totalBedrooms":834.0,"totalRooms":2902.0,"housingMedianAge":76.0,"medianHouseValue":111100.0,"medianIncome":2.12,"households":1000.0}]}'

In [19]:
headers = {
    'Content-Type': "application/json",
    'Authorization': "Bearer {}".format(token),
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.json())

{'rows': [{'latitude': 37.7944}, {'latitude': 37.922215}]}


## Images and tabular data

This is also a model of the California housing data, but with Google maps images added as an input and where we are instead predicting the mean house value.

In [22]:
url = ' -- insert the URL you find on the deployment page -- '
token = ' -- insert the token you find on the deployment page --'

### Encode images

We will re-use the `encode_imgs()` function defined above here.

In [23]:
img_files = ['images/15_5256_12656.png', 'images/15_5258_12653.png']
encoded_imgs = [encode_img(f) for f in img_files]

We can now populate the examples with numerical values and encoded images.

In [24]:
ex1 = {
"population": "1551.0",
"totalBedrooms": "434.0",
"totalRooms": "2202.0",
"housingMedianAge": "52.0",
"medianIncome": "3.12",
"households": "514.0",
"image_path": "\"" + encoded_imgs[0] + "\"",
"latitude": "37.88",
"longitude": "-122.25"
}

ex2 = {
"population": "3551.0",
"totalBedrooms": "834.0",
"totalRooms": "2902.0",
"housingMedianAge": "76.0",
"medianIncome": "2.12",
"households": "1000.0",
"image_path": "\"" + encoded_imgs[1] + "\"",
"latitude": "37.88",
"longitude": "-122.25"
}

examples = [ex1,ex2]
input_batch = ','.join([input_row(ex) for ex in examples])
payload = "{\"rows\": [" + input_batch + "]}"
headers = {
    'Content-Type': "application/json",
    'Authorization': "Bearer {}".format(token),
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.json())

{'rows': [{'medianHouseValue': 204714.05}, {'medianHouseValue': 298926.44}]}


## Image to image

Here we will send two images to a deployment and get two images back. The images come from the NoisyOffice dataset (https://archive.ics.uci.edu/ml/datasets/NoisyOffice) where the task is to clean images from stains and other imperfections.

In [63]:
url = ' -- insert the URL you find on the deployment page -- '
token = ' -- insert the token you find on the deployment page --'

In [64]:
img_files = ['images/FontLrm_Noisec_TE.png', 'images/FontLrm_Noisew_TE.png']
encoded_imgs = [encode_img(f) for f in img_files]
input_batch = ','.join(["{\"path_noisy\":\"" + encoded_img + "\"}" for encoded_img in encoded_imgs])
payload = "{\"rows\": [" + input_batch + "]}"
headers = {
    'Content-Type': "application/json",
    'Authorization': "Bearer {}".format(token),
    }

response = requests.request("POST", url, data=payload, headers=headers)

In [65]:
results = response.json()['rows']

Now you can, for example, save the generated images to file.

In [66]:
for i, res in enumerate(results):
    decoded = base64.b64decode(res['path_clean'].split(',')[-1])
    with open('images/image{}.png'.format(i), 'bw') as outf:
        outf = open('images/image{}.png'.format(i), 'bw')
    outf.write(decoded)